# A Better Center

This notebook demonstrates how to calculate the **"Pole of Inaccessibility"**, which may be a better notion of "center" for odd (especially concave) shapes.

Reference: https://en.wikipedia.org/wiki/Pole_of_inaccessibility

Let's start by loading an image from a GS bucket.
This will represent a supervoxel, for example.

In [ ]:
from google.cloud import storage
from PIL import Image
import numpy as np
import io
import matplotlib.pyplot as plt
from scipy.spatial import distance
from skimage.morphology import binary_dilation
from skimage.measure import label

def download_image_from_gcs(bucket_name, source_blob_name):
    """Downloads an image file from Google Cloud Storage and returns it as a PIL Image."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    image_data = blob.download_as_bytes()
    return Image.open(io.BytesIO(image_data))

# Specify the bucket and image path
bucket_name = "joe_exp"
source_blob_name = "shapes/shape_0.png"

# Download the image
image = download_image_from_gcs(bucket_name, source_blob_name)

# Convert the image to a NumPy array
image_array = np.array(image)

# Threshold the image at a value of 128, making a binary image (1 for the interior, 0 for the exterior)
threshold_value = 128
binary_image_array = (image_array < threshold_value).astype(np.uint8)

Now let's get a list of all the interior coordinates, and the perimeter coordinates.

In [ ]:
# Find the interior pixels (all pixels with a value of 1)
interior_coords = np.argwhere(binary_image_array == 1)

# Find the perimeter pixels
dilated_image = binary_dilation(binary_image_array)
perimeter_image = dilated_image ^ binary_image_array  # XOR to find perimeter pixels
perimeter_coords = np.argwhere(perimeter_image == 1)

# Convert the coordinates to lists of tuples
interior_coords = [tuple(coord) for coord in interior_coords]
perimeter_coords = [tuple(coord) for coord in perimeter_coords]

print("Number of interior pixels:", len(interior_coords))
print("Number of perimeter pixels:", len(perimeter_coords))

With the list of interior coordinates, calculating the centroid is trivial.
(It's just the average coordinate.)

In [ ]:
centroid = np.mean(interior_coords, axis=0)
print("Centroid:", centroid)

But now let's calculate the _Pole of Inaccessibility_ -- that is, the interior point
which is farthest away from the nearest perimeter point (so, most inaccessible).

In [ ]:
# Calculate the distance between each interior pixel and all perimeter pixels
distances = distance.cdist(interior_coords, perimeter_coords, metric='euclidean')

# Find the minimum distance to a perimeter pixel for each interior pixel
min_distances = distances.min(axis=1)

# Identify the interior pixel with the maximum of these minimum distances
max_min_distance_index = np.argmax(min_distances)
pole_of_inaccessibility = interior_coords[max_min_distance_index]

print("Pole of Inaccessibility:", pole_of_inaccessibility)

And now plot both types of center, so we can see.

In [ ]:
def show_result(binary_image_array, centroid, pole, legend_pos=(1, 0.5)):
    # Plot the binary image
    plt.imshow(binary_image_array, cmap='gray')
    #plt.title("Binary Image with Centroid and Pole of Inaccessibility")
    plt.axis('off')  # Hide the axis
    
    # Plot the centroid
    plt.scatter(*centroid[::-1], color='red', label='Centroid', marker='x', s=100)
    
    # Plot the pole of inaccessibility
    plt.scatter(*pole[::-1], color='blue', label='Pole of Inaccessibility', marker='o', s=100)
    
    if legend_pos is not None:
        plt.legend(loc='center left', bbox_to_anchor=legend_pos)

    # Adjust the layout to make space for the legend, and show it
    plt.tight_layout()
    plt.show()
show_result(binary_image_array, centroid, pole_of_inaccessibility)

## Let's Encapsulate

The above developed & demonstrated the algorithm step by step.  But now let's make a nice neat function to return the pole for any binary image (ndarray).  "Pole of inaccessibility" is an overly dramatic and wordy name, so we'll just call it the "center pole."

In [ ]:
def center_pole(binary_image_array):
    # Find the interior pixels (all pixels with a value of 1), and the surrounding perimiter pixels
    interior_coords = np.argwhere(binary_image_array == 1)

    # Find the perimeter pixels
    dilated_image = binary_dilation(binary_image_array)
    perimeter_image = dilated_image ^ binary_image_array  # XOR to find perimeter pixels
    perimeter_coords = np.argwhere(perimeter_image == 1)

    # Convert the coordinates to lists of tuples (needed by cdist)
    interior_coords = [tuple(coord) for coord in interior_coords]
    perimeter_coords = [tuple(coord) for coord in perimeter_coords]

    # Calculate the distance between each interior pixel and all perimeter pixels
    distances = distance.cdist(interior_coords, perimeter_coords, metric='euclidean')
    
    # Find the minimum distance to a perimeter pixel for each interior pixel
    min_distances = distances.min(axis=1)
    
    # Identify the interior pixel with the maximum of these minimum distances
    max_min_distance_index = np.argmax(min_distances)
    pole_of_inaccessibility = interior_coords[max_min_distance_index]
    return pole_of_inaccessibility

In [ ]:
def centroid(binary_image_array):
    # Find the interior pixels (all pixels with a value of 1),
    # and average to find the centroid
    interior_coords = np.argwhere(binary_image_array == 1)
    centroid = np.mean(interior_coords, axis=0)
    return centroid

And let's demonstrate it on a few more examples.

In [ ]:
def load_binary(source_blob_name, threshold_value=128):
    image = download_image_from_gcs(bucket_name, source_blob_name)
    image_array = np.array(image)
    return (image_array < threshold_value).astype(np.uint8)

In [ ]:
for i in range(1, 5):
    img = load_binary(f'shapes/shape_{i}.png')
    show_result(img, centroid(img), center_pole(img))